In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
# Read data from Excel
year = 2018
main_site = 'London'
df = pd.read_excel('ublox_dlop.xlsx', sheet_name=str(year))
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Week 47,Entry,Exit,Hours,Status,Comment
1,NaN,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,NaN,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,NaN,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,NaN,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment


In [3]:
# Remove first column which does not contain any information
df.drop(axis=1, columns="Unnamed: 0" , inplace= True)
df.head()

,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Week 47,Entry,Exit,Hours,Status,Comment
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment


In [4]:
# Rename all columns for sake of convenience
df.columns = ['Day of Week', 'Entry', 'Exit', 'Hours', 'Status', 'Comment']
df.head(n=10)

,Day of Week,Entry,Exit,Hours,Status,Comment
0,Week 47,Entry,Exit,Hours,Status,Comment
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment
5,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment
6,Weekly Hours,NaN,NaN,1900-01-01 18:00:00,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN
8,Week 48,Entry,Exit,Hours,Status,Comment
9,Monday,08:50:00,17:50:00,09:00:00,AA,Some comment


In [5]:
# Remove rows that do not contain any information 
# (in this case, the ones that only contain sum of weekly worked hours)
df.dropna(axis=0, subset = ['Status'], inplace=True)
df.head(n=10)

,Day of Week,Entry,Exit,Hours,Status,Comment
0,Week 47,Entry,Exit,Hours,Status,Comment
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment
5,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment
8,Week 48,Entry,Exit,Hours,Status,Comment
9,Monday,08:50:00,17:50:00,09:00:00,AA,Some comment
10,Tuesday,08:50:00,17:20:00,08:30:00,BB,Some comment
11,Wednesday,09:00:00,18:00:00,09:00:00,CC (Lisbon),Some comment


In [6]:
# Extract row indexes that contain the word Week (which means that they only contain the column title)
index_weekYear = df['Day of Week'].str.contains('Week')
index_weekYear.head()

0     True
1    False
2    False
3    False
4    False
Name: Day of Week, dtype: bool

In [7]:
# Get all Weeks of Year present in the datasheet
weekYear = df[index_weekYear]['Day of Week'].str.extract('(\d+)').values.flatten().astype(int)
weekYear

array([47, 48, 49, 50, 51, 52])

In [8]:
# Remove row indexes that contain the word Week
df.drop(df[index_weekYear].index, inplace=True)
df.head(n=10)

,Day of Week,Entry,Exit,Hours,Status,Comment
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment
5,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment
9,Monday,08:50:00,17:50:00,09:00:00,AA,Some comment
10,Tuesday,08:50:00,17:20:00,08:30:00,BB,Some comment
11,Wednesday,09:00:00,18:00:00,09:00:00,CC (Lisbon),Some comment
12,Thursday,09:00:00,17:00:00,08:00:00,BB (Lisbon),Some comment
13,Friday,09:00:00,16:00:00,07:00:00,BB (Lisbon),Some comment


In [9]:
# Repeat Week of year number to attribute to weekly days (Monday, Tuesday, Wednesday, Thursday, Friday)
def repeatWeekOfYear( weeksYear ):
    weekYearInWeekDays = np.array([])
    for weekOfYear in weeksYear: 
        weekYearInWeekDays = np.concatenate((weekYearInWeekDays, [weekOfYear]*5), axis=None)
    return(weekYearInWeekDays)

In [10]:
# Create column with Week of Year
repeatedWeekYear = repeatWeekOfYear(weekYear)
df['Week of Year'] = repeatedWeekYear[:len(df.index)].astype(int)
df.tail()

,Day of Week,Entry,Exit,Hours,Status,Comment,Week of Year
41,Monday,NaN,NaN,00:00:00,DAY OFF,NaN,52
42,Tuesday,NaN,NaN,00:00:00,BANK HOLIDAY,NaN,52
43,Wednesday,NaN,NaN,00:00:00,BANK HOLIDAY,NaN,52
44,Thursday,NaN,NaN,00:00:00,DAY OFF,NaN,52
45,Friday,NaN,NaN,00:00:00,DAY OFF,NaN,52


In [11]:
# Add column with Year
df['Year'] = year
df.head()

,Day of Week,Entry,Exit,Hours,Status,Comment,Week of Year,Year
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment,47,2018
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment,47,2018
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment,47,2018
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment,47,2018
5,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment,47,2018


In [12]:
# Panda Series with Day of Week
DayOfWeek = pd.Series([1, 2, 3, 4, 5], 
                      index = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
DayOfWeek

Monday       1
Tuesday      2
Wednesday    3
Thursday     4
Friday       5
dtype: int64

In [13]:
# Extract work date based on year, week of year and day of week
df['Date'] = [dt.datetime.strptime(year+' '+weekYear+' '+DayOfWeek[dayWeek].astype(str), "%Y %W %w")
 for year, weekYear, dayWeek
 in zip(df['Year'].values.astype(str), 
        (df['Week of Year'].values-1).astype(str), 
        df['Day of Week'].values.astype(str))];
df.head()

,Day of Week,Entry,Exit,Hours,Status,Comment,Week of Year,Year,Date
1,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment,47,2018,2018-11-12
2,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment,47,2018,2018-11-13
3,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment,47,2018,2018-11-14
4,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment,47,2018,2018-11-15
5,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment,47,2018,2018-11-16


In [14]:
# Change column order to be more intuitive
df = df[['Year', 'Week of Year', 'Date', 'Day of Week', 'Entry', 'Exit', 'Hours', 'Status', 'Comment']]
df.tail(n=7)

,Year,Week of Year,Date,Day of Week,Entry,Exit,Hours,Status,Comment
36,2018,51,2018-12-13,Thursday,09:50:00,18:40:00,08:50:00,AA,Some comment
37,2018,51,2018-12-14,Friday,10:00:00,16:00:00,06:00:00,AA,Some comment
41,2018,52,2018-12-17,Monday,NaN,NaN,00:00:00,DAY OFF,NaN
42,2018,52,2018-12-18,Tuesday,NaN,NaN,00:00:00,BANK HOLIDAY,NaN
43,2018,52,2018-12-19,Wednesday,NaN,NaN,00:00:00,BANK HOLIDAY,NaN
44,2018,52,2018-12-20,Thursday,NaN,NaN,00:00:00,DAY OFF,NaN
45,2018,52,2018-12-21,Friday,NaN,NaN,00:00:00,DAY OFF,NaN


In [15]:
# Fill Entry and Exit times (useful when recreating hour column)
df['Entry'].fillna(dt.datetime.strptime('0:0:0', '%H:%M:%S').time(), inplace = True)
df['Exit'].fillna(dt.datetime.strptime('0:0:0', '%H:%M:%S').time(), inplace = True)
df.head()

,Year,Week of Year,Date,Day of Week,Entry,Exit,Hours,Status,Comment
1,2018,47,2018-11-12,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment
2,2018,47,2018-11-13,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment
3,2018,47,2018-11-14,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment
4,2018,47,2018-11-15,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment
5,2018,47,2018-11-16,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment


In [16]:
# Create Location column (When abroad it's written inside parenthesis in Status, otherwise in main site)
df['Location'] = df['Status'].str.extract('.*\((.*)\).*',expand=True)
df['Location'].fillna(main_site, inplace = True)

In [17]:
# Remove content that is in parentheses (thus, abroad location site if existent)
df['Status'] = df['Status'].str.extract('([a-z-A-Z ]+)', expand=False).str.strip()
df.head()

,Year,Week of Year,Date,Day of Week,Entry,Exit,Hours,Status,Comment,Location
1,2018,47,2018-11-12,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment,London
2,2018,47,2018-11-13,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment,London
3,2018,47,2018-11-14,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment,London
4,2018,47,2018-11-15,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment,London
5,2018,47,2018-11-16,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment,London


In [18]:
# Select Working date as index and remove unnecessary Date and Year column
df.index = df['Date']
del df['Date']
del df['Year']
df.head()

,Week of Year,Day of Week,Entry,Exit,Hours,Status,Comment,Location
Date,,,,,,,,
2018-11-12,47,Monday,10:00:00,18:30:00,08:30:00,AA,Some comment,London
2018-11-13,47,Tuesday,09:00:00,17:30:00,08:30:00,AA,Some comment,London
2018-11-14,47,Wednesday,08:45:00,17:45:00,09:00:00,AA,Some comment,London
2018-11-15,47,Thursday,08:45:00,16:45:00,08:00:00,AA,Some comment,London
2018-11-16,47,Friday,09:15:00,17:15:00,08:00:00,AA,Some comment,London


In [19]:
# Recreate hour column to float instead of datetime.time object
for date in df.index:
        df.loc[date,'Hours'] = (round(((df['Exit'][date].hour-df['Entry'][date].hour) + \
                                  (df['Exit'][date].minute-df['Entry'][date].minute)/60),2))
df.head()

,Week of Year,Day of Week,Entry,Exit,Hours,Status,Comment,Location
Date,,,,,,,,
2018-11-12,47,Monday,10:00:00,18:30:00,8.5,AA,Some comment,London
2018-11-13,47,Tuesday,09:00:00,17:30:00,8.5,AA,Some comment,London
2018-11-14,47,Wednesday,08:45:00,17:45:00,9,AA,Some comment,London
2018-11-15,47,Thursday,08:45:00,16:45:00,8,AA,Some comment,London
2018-11-16,47,Friday,09:15:00,17:15:00,8,AA,Some comment,London
